## setup

In [ ]:
# Setup
!pip install -q wordcloud
!pip install -U sentence-transformers
!pip install annoy

import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string

import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
                 
import pickle
from sentence_transformers import SentenceTransformer

from annoy import AnnoyIndex

import json

from six import string_types

import tqdm

     |████████████████████████████████| 81kB 3.4MB/s 
     |████████████████████████████████| 2.1MB 10.8MB/s 
     |████████████████████████████████| 1.2MB 40.6MB/s 
     |████████████████████████████████| 3.3MB 50.3MB/s 
     |████████████████████████████████| 901kB 42.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=d65deafd9637293973ec408f64ab8043093f07bec5c10d9cce5b7d84c9761aa3
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers
     |████████████████████████████████| 655kB 6.5MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391598 sha256=3ebfb5d13aa5978f55bd210be4c5ef047cdec9c11baaefb37a970d9a9cd015ff
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy
[nltk_data] Downloading package stopwords to

In [ ]:
!nvidia-smi

Fri May  7 21:45:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    34W /  70W |   1774MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## load

In [ ]:
!ls 
!wget https://ai2-public-datasets.s3.amazonaws.com/open-book-qa/OpenBookQA-V1-Sep2018.zip

sample_data
--2021-05-07 21:46:56--  https://ai2-public-datasets.s3.amazonaws.com/open-book-qa/OpenBookQA-V1-Sep2018.zip
Resolving ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)... 52.218.153.83
Connecting to ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)|52.218.153.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1446098 (1.4M) [binary/octet-stream]
Saving to: ‘OpenBookQA-V1-Sep2018.zip’

OpenBookQA-V1-Sep20 100%[===================>]   1.38M  3.79MB/s    in 0.4s    

2021-05-07 21:46:56 (3.79 MB/s) - ‘OpenBookQA-V1-Sep2018.zip’ saved [1446098/1446098]



In [ ]:
!ls 
!unzip OpenBookQA-V1-Sep2018.zip && ls

OpenBookQA-V1-Sep2018.zip  sample_data
Archive:  OpenBookQA-V1-Sep2018.zip
   creating: OpenBookQA-V1-Sep2018/
   creating: OpenBookQA-V1-Sep2018/Data/
   creating: OpenBookQA-V1-Sep2018/Data/Additional/
  inflating: OpenBookQA-V1-Sep2018/Data/Additional/test_complete.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Additional/train_complete.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Additional/crowdsourced-facts.txt  
  inflating: OpenBookQA-V1-Sep2018/Data/Additional/dev_complete.jsonl  
   creating: OpenBookQA-V1-Sep2018/Data/Main/
  inflating: OpenBookQA-V1-Sep2018/Data/Main/train.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/test.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/train.tsv  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/dev.tsv  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/dev.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/openbook.txt  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/test.tsv  
OpenBookQA-V1-Sep2018  OpenBookQA-V1-Sep2018.zip  sam

In [ ]:
!cd OpenBookQA-V1-Sep2018/Data && ls && pwd

Additional  Main
/content/OpenBookQA-V1-Sep2018/Data


In [ ]:
f = open('/content/OpenBookQA-V1-Sep2018/Data/Additional/crowdsourced-facts.txt', 'r+')
data1 = pd.read_csv(f, sep='\n', header=None, names=['facts'])
f.close()
f = open('/content/OpenBookQA-V1-Sep2018/Data/Main/openbook.txt', 'r+')
data2 = pd.read_csv(f, header=None, names=['facts'])
f.close()

print(data1)
print("--------------")
print(data2)
print("--------------")
frames = [data1, data2]
data = pd.concat(frames)
print(data)
print("--------------")

# Convert to list
li_data1 = data1['facts'].tolist()
li_data2 = data2['facts'].tolist()
li_data = data['facts'].tolist()
print(li_data1)
print("--------------")
print(li_data2)
print("--------------")
print(li_data)

                                                  facts
0                                                -2+2=0
1                                1 day has 1440 minutes
2     1 out of 4 people in the world are chronically...
3     100 degrees celsius is the boiling point of water
4                                      12:00 PM is noon
...                                                 ...
5161                                    zero is nothing
5162                        zipping powered is electric
5163        zooplankton do not have respiration systems
5164            zoos are public exhibitions for viewing
5165                zoos hold animals outside of nature

[5166 rows x 1 columns]
--------------
                                                  facts
0                         A bee is a pollinating animal
1                        A bird is a pollinating animal
2     An electrical conductor is a vehicle for the f...
3     An example of a change in the Earth is an ocea...
4     An

## Test tokenize

In [ ]:
# Constants
# POS (Parts Of Speech) for: nouns, adjectives, verbs and adverbs
DI_POS_TYPES = {'NN':'n', 'JJ':'a', 'VB':'v', 'RB':'r'} 
POS_TYPES = list(DI_POS_TYPES.keys())

# Constraints on tokens
MIN_STR_LEN = 3
RE_VALID = '[a-zA-Z]'

In [ ]:
# Get stopwords, stemmer and lemmatizer
stopwords = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# Remove accents function
def remove_accents(data):
    return ''.join(x for x in unicodedata.normalize('NFKD', data) if x in string.ascii_letters or x == " ")

def tokenize_data(li_data):
    # Process all quotes
    li_tokens = []
    li_token_lists = []
    li_lem_strings = []

    for i,text in enumerate(li_data):
        # Tokenize by sentence, then by lowercase word
        tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]

        # Process all tokens per fact
        li_tokens_fact = []
        li_tokens_fact_lem = []
        for token in tokens:
            # Remove accents
            t = remove_accents(token)

            # Remove punctuation
            t = str(t).translate(string.punctuation)
            li_tokens_fact.append(t)
            
            # Add token that represents "no lemmatization match"
            li_tokens_fact_lem.append("-") # this token will be removed if a lemmatization match is found below

            # Process each token
            if t not in stopwords:
                if re.search(RE_VALID, t):
                    if len(t) >= MIN_STR_LEN:
                        # Note that the POS (Part Of Speech) is necessary as input to the lemmatizer 
                        # (otherwise it assumes the word is a noun)
                        pos = nltk.pos_tag([t])[0][1][:2]
                        pos2 = 'n'  # set default to noun
                        if pos in DI_POS_TYPES:
                          pos2 = DI_POS_TYPES[pos]
                        
                        stem = stemmer.stem(t)
                        lem = lemmatizer.lemmatize(t, pos=pos2)  # lemmatize with the correct POS
                        
                        if pos in POS_TYPES:
                            li_tokens.append((t, stem, lem, pos))

                            # Remove the "-" token and append the lemmatization match
                            li_tokens_fact_lem = li_tokens_fact_lem[:-1] 
                            li_tokens_fact_lem.append(lem)

        # Build list of token lists from lemmatized tokens
        li_token_lists.append(li_tokens_fact)
        
        # Build list of strings from lemmatized tokens
        str_li_tokens_fact_lem = ' '.join(li_tokens_fact_lem)
        li_lem_strings.append(str_li_tokens_fact_lem)
        
    # Build resulting dataframes from lists
    df_token_lists = pd.DataFrame(li_token_lists)

    print("df_token_lists.head(5):")
    print(df_token_lists.head(5).to_string())
    
    # Replace None with empty string
    for c in df_token_lists:
        if str(df_token_lists[c].dtype) in ('object', 'string_', 'unicode_'):
            df_token_lists[c].fillna(value='', inplace=True)

    df_lem_strings = pd.DataFrame(li_lem_strings, columns=['lem fact'])

    print()
    print("")
    print("df_lem_strings.head():")
    print(df_lem_strings.head().to_string())

    return df_token_lists, df_lem_strings

data_token_lists, data_lem_strings = tokenize_data(li_data)
print("---------")
print(data_token_lists)
print("--------------")
print(data_lem_strings)
print("--------------")

df_token_lists.head(5):
  0        1        2     3        4        5      6      7      8            9       10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28    29    30    31    32    33    34    35    36    37    38    39
0        None     None  None     None     None   None   None   None         None    None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None
1         day      has        minutes     None   None   None   None         None    None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None
2         out       of         people       in    the  world    are  chronically  hungry        None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  N

## TfidfVectorizer Test

In [ ]:
v = TfidfVectorizer()
x = v.fit_transform(data_lem_strings['lem fact'])
print(x.toarray())
print(x.toarray().shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6492, 4166)


In [ ]:
v = TfidfVectorizer()
x = v.fit_transform(data['facts'])
print(x.toarray())
print(x.toarray().shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6492, 5689)


#### WIP


In [ ]:
# nevermind this for now...
vectors_for_training = x.toarray() #get the vectors back out of the dataframe for use in something else
fact_vector = data['facts'].tolist()
print(vectors_for_training.shape)
X_train, y_train, X_test, y_test = train_test_split(vectors_for_training, fact_vector)

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(df.tfidf.tolist())

(6492, 5689)


ValueError: ignored

## SentenceTransformer Test

### common functions

In [ ]:
def load_embeddings():
    return pickle.load(open("embeddings.pkl", "rb"))

def load_sentence_ids():
    return pickle.load(open("sentence_ids.pkl", "rb"))

def get_embeddings_dim(embeddings):
    dim = None
    for embedding in embeddings.values():
        new_dim = len(embedding)
        if dim is None:
            dim = new_dim
        else:
            assert(new_dim == dim)  # All embeddings have the same dimention.
    return dim

### adapt dataset

In [ ]:
def adapt_dataset(f):
    all_data = []
    all_ids = set()
    for line in f:
        entry = json.loads(line)
        #assert(len(entry) == 3)

        question_id = entry["id"]
        correct_answer = entry["answerKey"]
        assert(isinstance(question_id, string_types))
        assert(correct_answer in ["A", "B", "C", "D"])

        entry = entry["question"]
        assert(len(entry) == 2)
        question = entry["stem"]
        answers = [
            choice["text"]
            for choice in sorted(
                entry["choices"],
                key=lambda x: x["label"])
        ]
        assert(isinstance(question, string_types))
        assert(len(answers) == 4)
        for answer in answers:
            assert(isinstance(answer, string_types))
        assert(question_id not in all_ids)
        all_ids.add(question_id)
        new_row = {
            "id": question_id,
            "question": question,
            "answers": answers,
            "correct": correct_answer
        }
        all_data.append(new_row)
    assert(len(all_data) == len(all_ids))
    df = pd.DataFrame(all_data)
    return df
#    print(train_df)

f = open('/content/OpenBookQA-V1-Sep2018/Data/Additional/train_complete.jsonl', 'rt')
train_df = adapt_dataset(f)
f.close()

f = open('/content/OpenBookQA-V1-Sep2018/Data/Additional/dev_complete.jsonl', 'rt')
dev_df = adapt_dataset(f)
f.close()

f = open('/content/OpenBookQA-V1-Sep2018/Data/Additional/test_complete.jsonl', 'rt')
test_df = adapt_dataset(f)
f.close()

### Create embeddings for Dataset

In [ ]:
def get_all_sentences():
    sentences = list(OBQAFacts())
    df = pd.concat(map(dm.ALLEN_AI_OBQA, list(OBQAType)))
    for _, row in df.iterrows():
        sentences.append(row.question)
        for answer in row.answers:
            sentences.append(row.question + " " + answer)
        sentences.append(row.question + " " + ", ".join(row.answers))
    return list(set(sentences))
    
#model = SentenceTransformer("roberta-large-nli-mean-tokens")
model = SentenceTransformer("msmarco-roberta-base-v3")
sentences = list(set(data['facts']))
print(len(sentences))
df = train_df
for _, row in df.iterrows():
    sentences.append(row.question)
    for answer in row.answers:
        sentences.append(row.question + " " + answer)
    sentences.append(row.question + " " + ", ".join(row.answers))
print(len(sentences))
df = dev_df
for _, row in df.iterrows():
    sentences.append(row.question)
    for answer in row.answers:
        sentences.append(row.question + " " + answer)
    sentences.append(row.question + " " + ", ".join(row.answers))
print(len(sentences))
df = test_df
for _, row in df.iterrows():
    sentences.append(row.question)
    for answer in row.answers:
        sentences.append(row.question + " " + answer)
    sentences.append(row.question + " " + ", ".join(row.answers))
sentence_embeddings = model.encode(sentences, show_progress_bar=True)
print(len(sentences))
assert(len(sentence_embeddings) == len(sentences))
data_emb = {}
for sentence, embedding in zip(sentences, sentence_embeddings):
    data_emb[sentence] = embedding
pickle.dump(data_emb, open("embeddings.pkl", "wb"))
print("Embeddings written to embeddings.pkl")


6487
36229
39229


KeyboardInterrupt: ignored

In [ ]:
print(len(list(set(data['facts']))))
print(len(sentences))
print(len(sentences))

6487
42229
42229


In [ ]:
!mv sentence_ids.pkl /content/drive/MyDrive/sentence_ids.pkl

### create index

In [ ]:
embeddings = load_embeddings()
print("Found {} embeddings.".format(len(embeddings)))

# Annoy uses Euclidean distance of normalized vectors for its angular
# distance, which for two vectors u,v is equal to sqrt(2(1-cos(u,v))).
index = AnnoyIndex(get_embeddings_dim(embeddings), "angular")
sentence_ids = {}
for idx, fact in enumerate(data['facts']):
    index.add_item(idx, embeddings[fact])
    sentence_ids[idx] = fact
index.build(1024)  # N trees
index.save('index.ann')
pickle.dump(sentence_ids, open("sentence_ids.pkl", "wb"))
print("Index saved to index.ann")
print("Sentence IDs saved to sentence_ids.pkl")

Found 41690 embeddings.
Index saved to index.ann
Sentence IDs saved to sentence_ids.pkl


### WIP: testing Results, annotate question db for downstream task 

In [ ]:
embeddings = load_embeddings()
sentence_ids = load_sentence_ids()
index = AnnoyIndex(get_embeddings_dim(embeddings), "angular")
index.load("index.ann")


def get_similar_sentences(query, k):
#    embeddings = load_embeddings()
#    sentence_ids = load_sentence_ids()
#    index = AnnoyIndex(get_embeddings_dim(embeddings), "angular")
#    index.load("index.ann")
#    print("Found {} items in the index.".format(index.get_n_items()))
#    print("The index uses {} trees.".format(index.get_n_trees()))
#    print("")
    if query in embeddings:
        closest, dists = index.get_nns_by_vector(embeddings[query], k, include_distances=True)  # noqa: E501
    else:
        q_embed = model.encode(query, show_progress_bar=True)
        closest, dists = index.get_nns_by_vector(q_embed, k, include_distances=True)  # noqa: E501
#    assert(len(closest) == len(dists))
    closest = map(lambda sid: sentence_ids[sid], closest)
    return zip(closest, dists)

def by_sentence(query):
    similar = get_similar_sentences(query)
    print("Query: `{}`".format(query))
    for i, (sentence, dist) in enumerate(similar):
        print("{}) {} ({})".format(i, sentence, dist))

def get_query_facts(sample, k):
#    facts = []
    similar_all = []    
#    similar_filtered = []      
    unique_facts = []   
    # Todo: this i_ variable is shit, find better solution  
#    i_ = []
#    _question = sample["question"]['stem']
#    for idx in range(len(sample['question']['choices'])):
#        similar = get_similar_sentences(_question + sample["question"]['choices'][idx]['text'])


#    train_samples = []
#    with open('/content/OpenBookQA-V1-Sep2018/Data/Additional/train_complete.jsonl') as f:
#        for item in json_lines.reader(f):
#            train_samples.append(item)

#    sample = train_samples[23]
    _question = sample["question"]['stem']

#    print("Question: " + _question + "\n")
#-------------------------------------------------------------------

#    for idx in range(len(sample['question']['choices'])): 
#        similar = get_similar_sentences(_question + ' ' + sample["question"]['choices'][idx]['text'])
#        print("{}) {}".format(idx, sample["question"]['choices'][idx]['text']))
#        for i, (sentence, dist) in enumerate(similar):
#            similar_all.append([idx, [sentence, dist]])

#-------------------------------------------------------------------
    similar = get_similar_sentences(_question, k)

#-------------------------------------------------------------------


#    df = train_df.sample(n=1)
#    row = next(df.iterrows())[1]
#    print("Question: " + row.question + "\n")
#    for idx, answer in zip(["A", "B", "C", "D"], row.answers):
#        print("{}) {}".format(idx, answer))
#    print("")
#    print("Answer: " + row.correct + "\n")
#    print("---------------------------------")
#   
#    for idx, answer in zip(["A", "B", "C", "D"], row.answers):
#        similar = get_similar_sentences(row.question + ' ' + answer)
#        for i, (sentence, dist) in enumerate(similar):
#            similar_all.append([idx, [sentence, dist]])
## Todo:
#        j = 0
#        for i, (sentence, dist) in enumerate(similar):
#            if sentence not in unique_facts:
#                facts.append(sentence)
#                j += 1
#            if j>3:
#                break
##
        
#        i_.append(0)

#    for idx in range(len(sample['question']['choices'])):
#        for i, (sentence, dist) in enumerate(similar_all[idx]):
#            for element in unique_facts:
#                if element[1][0] == sentence:
#                    if element[1] >
#            if sentence not in unique_facts:##-------------------------------
#                unique_facts.append(similar_all[idx][i])
#            else:
#                if dist < unique_facts.index(sentence)
#            print("{}) {} ({})".format(i, sentence, dist))
#            i_[idx] += 1
#            if i_[idx] > 4:
#                break; 
#    similar_all.sort(key = lambda x: x[1][1])
#    print(similar_all)
#    print("---------------------------------")
     
#    i_ = {"A":0,"B":0,"C":0,"D":0}
#    i_ = [0,0,0,0]
#    i = 0 
#    for fact in similar_all:
#        if i > 9:
#            break
#        if i_[fact[0]] > 2:
#            continue
#        if fact[1][0] not in unique_facts:
#            unique_facts.append(fact[1][0])
#            i_[fact[0]] += 1
#            i += 1

    for i, (sentence, dist) in enumerate(similar):
        unique_facts.append(sentence)
    
#    print(unique_facts)
#    print("---------------------------------")
    return unique_facts

def get_string_facts(s,k):  
    unique_facts = []   
    similar = get_similar_sentences(s,k)
    for i, (sentence, dist) in enumerate(similar):
        unique_facts.append(sentence)
    return unique_facts

def by_random_question():
    df = train_df.sample(n=1)
    row = next(df.iterrows())[1]
    print("Question: " + row.question + "\n")
    for idx, answer in zip(["A", "B", "C", "D"], row.answers):
        print("{}) {}".format(idx, answer))
    print("")
    print("Answer: " + row.correct + "\n")
    similar = get_string_facts(row.question + ' ' + row.answers[0], 4)
    print("Similar facts A:")
    for i, sentence in enumerate(similar):
        print("{}) {}".format(i, sentence))
    similar = get_string_facts(row.question + ' ' + row.answers[1], 4)
    print("Similar facts B:")
    for i, sentence in enumerate(similar):
        print("{}) {}".format(i, sentence))
    similar = get_string_facts(row.question + ' ' + row.answers[2], 4)
    print("Similar facts C:")
    for i, sentence in enumerate(similar):
        print("{}) {}".format(i, sentence))
    similar = get_string_facts(row.question + ' ' + row.answers[3], 4)
    print("Similar facts D:")
    for i, sentence in enumerate(similar):
        print("{}) {}".format(i, sentence))
    

def annotate_all_questions():
    embeddings = load_embeddings()
    sentence_ids = load_sentence_ids()
    index = AnnoyIndex(get_embeddings_dim(embeddings), "angular")
    index.load("index.ann")
    print("Found {} items in the index.".format(index.get_n_items()))
    print("The index uses {} trees.".format(index.get_n_trees()))
    print("")

    df = train_df
    annotations = {}
    for _, row in tqdm.tqdm(df.iterrows(), total=len(df)):
        for answer in row.answers:
            sent = row.question + " " + answer
            closest = index.get_nns_by_vector(embeddings[sent], 75)
            closest = list(map(lambda sid: sentence_ids[sid], closest))
            annotations[sent] = closest
    pickle.dump(annotations, open("annotations.pkl", "wb"))
    print("Annotations written to annotations.pkl")


# by_sentence("Pasta may be cooked in water when")
# by_sentence("what is the closest source of plasma to our planet?")
# by_sentence("If an organism is existing then it is made up of")
by_random_question()
# annotate_all_questions()

# get_query_facts(1)

Question: One of Kangaroo's unusual features are

A) They have horns
B) They speak English
C) They can fly
D) webbed feet

Answer: D

Similar facts A:
0) hummingbirds have sharp beaks
1) falcons have sharp beaks
2) Kangaroos have webbed feet
3) Chickens have feathers
Similar facts B:
0) An example of an instinct is the kangaroo 's ability to crawl into its mother 's pouch to drink milk
1) Kangaroos have webbed feet
2) animals learn some behaviors from watching their parents
3) different birds eat different food, so beaks need to be different
Similar facts C:
0) Kangaroos have webbed feet
1) as the weight of an animal decreases , that animal will fly more easily
2) birds get off the ground by flying
3) falcons have sharp beaks
Similar facts D:
0) Kangaroos have webbed feet
1) flamingos have webbed feet
2) robins don't have webbed feet
3) webbed feet are used for moving faster through water by aquatic animals


# random test stuff

In [ ]:
# Setup
!pip install -q wordcloud
import wordcloud

import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.decomposition import PCA
from sklearn.manifold import MDS

pd.set_option("display.precision", 4)

# Load toy dataset
dat = [
         ['X', 'aaa aaa'],
         ['Y', 'aaa bbb bbb'],
         ['Y', 'aaa bbb bbb ddd'],
         ['Z', 'aaa bbb ccc eee'],
         ['Z', 'aaa bbb ccc fff']
      ]
df_sentences = pd.DataFrame(dat, columns=['Label', 'Sentence'])

labels = df_sentences['Label']
print("labels:")
print(labels)

n_clusters = np.unique(labels).shape[0]
print("n_clusters:", n_clusters)

labels:
0    X
1    Y
2    Y
3    Z
4    Z
Name: Label, dtype: object
n_clusters: 3


# Albert OpenbookQA
Author: Siyu Chen 

Messing around with it: Philippe Schwarz


## Setup

### Install dependencies

In [ ]:
# use tpu in pytorch
!pip install --quiet cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 144.6MB 103kB/s 
     |████████████████████████████████| 61kB 2.4MB/s 
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.


In [ ]:
!pip install --quiet lineflow
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet json_lines

# Albert requires SentencePiece
!pip install --quiet SentencePiece

     |████████████████████████████████| 808kB 5.4MB/s 
     |████████████████████████████████| 829kB 9.2MB/s 
     |████████████████████████████████| 112kB 24.7MB/s 
     |████████████████████████████████| 645kB 17.8MB/s 
     |████████████████████████████████| 276kB 26.4MB/s 
     |████████████████████████████████| 1.3MB 22.5MB/s 
     |████████████████████████████████| 143kB 41.0MB/s 
     |████████████████████████████████| 296kB 40.1MB/s 
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.


### Import libraries which are needed for fine-tuning

In [ ]:
from typing import Dict
from pathlib import Path
import json
from functools import partial
from collections import OrderedDict
from argparse import ArgumentParser

import lineflow as lf
from transformers import AlbertForMultipleChoice, AlbertTokenizer, AdamW
import pytorch_lightning as pl

import torch
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
import json_lines

### Define constant variables

In [ ]:
MAX_LEN = 256
NUM_LABELS = 4
label_map = {"A": 0, "B": 1, "C": 2, "D": 3}
BATCH_SIZE = 32

## Dataset

### WIP Useful function to process dataset

In [ ]:
def raw_samples_to_dataset(samples):
    datas = []
    i = 0
    for sample in samples:
        i+=1
        if i%250 == 0:
            print(i)
        _id = sample["id"]
        _article = ""
        for fact in get_query_facts(sample, 5):
          _article = _article + fact + ". "
        _question = sample["question"]['stem']
        _options = []
        _answer = sample["answerKey"]
        for idx in range(len(sample['question']['choices'])): 
            _options.append(sample["question"]['choices'][idx]['text'])

        data = {
                "id": _id,
                "article": _article,
                "options": _options,
                "question": _question,
                "answer": _answer
                }
        datas.append(data)
#        print(".")
    return lf.Dataset(datas)


def preprocess(tokenizer: AlbertTokenizer, x: Dict) -> Dict:

    choices_features = []

    option: str
    for option in x["options"]:
        text_a = x["article"]
        text_b = x["question"] + " " + option

        inputs = tokenizer.encode_plus(
                text_a,
                text_b,
                add_special_tokens=True,
                max_length=MAX_LEN
                )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
        attention_mask = [1] * len(input_ids)

        pad_token_id = tokenizer.pad_token_id
        padding_length = MAX_LEN - len(input_ids)
        input_ids = input_ids + ([pad_token_id] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([pad_token_id] * padding_length)

        assert len(input_ids) == MAX_LEN, "Error with input length {} vs {}".format(len(input_ids), MAX_LEN)
        assert len(attention_mask) == MAX_LEN, "Error with input length {} vs {}".format(len(attention_mask), MAX_LEN)
        assert len(token_type_ids) == MAX_LEN, "Error with input length {} vs {}".format(len(token_type_ids), MAX_LEN)

        choices_features.append({
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            })

    labels = label_map.get(x["answer"], -1)
    label = torch.tensor(labels).long()

    return {
            "id": x["id"],
            "label": label,
            "input_ids": torch.tensor([cf["input_ids"] for cf in choices_features]),
            "attention_mask": torch.tensor([cf["attention_mask"] for cf in choices_features]),
            "token_type_ids": torch.tensor([cf["token_type_ids"] for cf in choices_features]),
            }


def get_dataloader(tokenizer, cachedir: str = "./"):
    cachedir = Path(cachedir)
    

    preprocessor = partial(preprocess, tokenizer)

    train_file_name = "train_openbook.cache"
    train_path = Path(cachedir / train_file_name)
    if train_path.exists():
        print(f'Loading data from {train_file_name}...')
        with train_path.open('rb') as f:
            train_cache = pickle.load(f)

    train_dataloader = DataLoader(
            lf.core.CacheDataset(train_cache),
            batch_size=BATCH_SIZE
            )

    val_file_name = "val_openbook.cache"
    val_path = Path(cachedir / val_file_name)
    if val_path.exists():
        print(f'Loading data from {val_file_name}...')
        with val_path.open('rb') as f:
            val_cache = pickle.load(f)

    val_dataloader = DataLoader(
            lf.core.CacheDataset(val_cache),
            batch_size=BATCH_SIZE
            )

    
    test_file_name = "test_openbook.cache"
    test_path = Path(cachedir / test_file_name)
    if test_path.exists():
        print(f'Loading data from {test_file_name}...')
        with test_path.open('rb') as f:
            test_cache = pickle.load(f)

    test_dataloader = DataLoader(
            lf.core.CacheDataset(test_cache),
            batch_size=BATCH_SIZE
            )

    return train_dataloader, val_dataloader, test_dataloader

### Download OpenBook dataset and unzip dataset

#### Download dataset

In [ ]:
!ls 
!wget https://ai2-public-datasets.s3.amazonaws.com/open-book-qa/OpenBookQA-V1-Sep2018.zip

embeddings.pkl	OpenBookQA-V1-Sep2018	   sample_data
index.ann	OpenBookQA-V1-Sep2018.zip  sentence_ids.pkl
--2021-05-07 21:10:31--  https://ai2-public-datasets.s3.amazonaws.com/open-book-qa/OpenBookQA-V1-Sep2018.zip
Resolving ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)... 52.218.237.219
Connecting to ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)|52.218.237.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1446098 (1.4M) [binary/octet-stream]
Saving to: ‘OpenBookQA-V1-Sep2018.zip.1’

OpenBookQA-V1-Sep20 100%[===================>]   1.38M  2.89MB/s    in 0.5s    

2021-05-07 21:10:32 (2.89 MB/s) - ‘OpenBookQA-V1-Sep2018.zip.1’ saved [1446098/1446098]



#### Unzip dataset

In [ ]:
!ls 
!unzip OpenBookQA-V1-Sep2018.zip && ls 

embeddings.pkl	       OpenBookQA-V1-Sep2018.zip    sentence_ids.pkl
index.ann	       OpenBookQA-V1-Sep2018.zip.1
OpenBookQA-V1-Sep2018  sample_data
Archive:  OpenBookQA-V1-Sep2018.zip
replace OpenBookQA-V1-Sep2018/Data/Additional/test_complete.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
embeddings.pkl	       OpenBookQA-V1-Sep2018.zip    sentence_ids.pkl
index.ann	       OpenBookQA-V1-Sep2018.zip.1
OpenBookQA-V1-Sep2018  sample_data


In [ ]:
!cd OpenBookQA-V1-Sep2018/Data/Additional && ls && pwd

crowdsourced-facts.txt	test_complete.jsonl
dev_complete.jsonl	train_complete.jsonl
/content/OpenBookQA-V1-Sep2018/Data/Additional


### Call functions to handle raw dataset

In [ ]:
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2", do_lower_case=True)
train_dataloader, val_dataloader, test_dataloader = get_dataloader(tokenizer, '/content/drive/My Drive/OpenBook/')

Loading data from train_openbook.cache...
Loading data from val_openbook.cache...
Loading data from test_openbook.cache...


In [ ]:
train_dataloader_small_batch_size = DataLoader(train_dataloader.dataset, batch_size=8)
val_dataloader_small_batch_size = DataLoader(val_dataloader.dataset, batch_size=8)
test_dataloader_small_batch_size = DataLoader(test_dataloader.dataset, batch_size=8)

In [ ]:
print(len(train_dataloader_small_batch_size))
print(len(val_dataloader_small_batch_size))
print(len(test_dataloader_small_batch_size))

620
63
63


## Play around with `DataLoader`

### Get a batch of dataloader

In [ ]:
sample = next(iter(test_dataloader_small_batch_size))

### Print some information of dataloader

In [ ]:
# type of sample
print(type(sample))
# keys of sample
print(sample.keys())
# ids of sample
print(sample['id'])
# label of sample
print(sample['label'])

<class 'dict'>
dict_keys(['id', 'label', 'input_ids', 'attention_mask', 'token_type_ids'])
['8-343', '1129', '880', '7-999', '8-464', '9-794', '9-1163', '9-322']
tensor([1, 0, 2, 2, 2, 2, 2, 1])


In [ ]:
# tokenised context and question
print(sample['input_ids'][0].size())
print(sample['input_ids'][0][0])

torch.Size([4, 256])
tensor([    2,   568,   787,  2566,   951,  4047,   875,    20,    44,  4377,
            9, 17936,  1108,  7599,     9,  7599,  4495, 12619,    19,  1065,
        13759,     9,   228,   647,  1108,    20,  2658,  4600,     9,  2079,
          308,    20,   233,    32,   236,     9,     3,    21,   840,  2846,
           20,   799,  7599,   875,    86,    30,    59,    92,  7829,    21,
         2210, 10057,    35,    14,   241,    16,    14,   159,     9,    75,
          699,    84,    66,  3391,    17, 12640,    15,    59,  4073,    14,
          246,   161,    20,  2079,   875,    25,    20,   233,    91,  1132,
         3029,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0, 

### Decode tokenised context and question

In [ ]:
de_tokenizer = tokenizer

In [ ]:
de_tokenizer.decode(sample['input_ids'][0][1])

'[CLS] using less resources usually causes money to be saved. preserving means saving. saving plastic bottles in conserving. going green means to avoid waste. save water to make it last.[SEP] a person wants to start saving money so that they can afford a nice vacation at the end of the year. after looking over their budget and expenses, they decide the best way to save money is to quit eating lunch out[SEP]<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

## Fine tuning on AlbertModel

### Set up connection to Google drive in order to save checkpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Define a model

In [ ]:
from pytorch_lightning.metrics import functional as FM
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
class Model(pl.LightningModule):

    def __init__(self):
        super(Model, self).__init__()

        model = AlbertForMultipleChoice.from_pretrained("albert-base-v2", num_labels=NUM_LABELS)
        self.model = model

        self._train_dataloader = train_dataloader_small_batch_size
        self._val_dataloader = val_dataloader_small_batch_size
        self._test_dataloader = test_dataloader_small_batch_size

    def configure_optimizers(self):
        no_decay = ['bias', 'LayerNorm.weight']
        weight_decay = 0.0
        adam_epsilon = 1e-8

        optimizer_grouped_parameters = [
            {
                'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                'weight_decay': weight_decay
                },
            {
                'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0,
                }
            ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=adam_epsilon)

        return optimizer

    def training_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        outputs = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
                )
        
        self.log('train_loss', outputs.loss, on_epoch=True, on_step=True, prog_bar=True, logger=True)
        return outputs.loss

    def validation_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        outputs = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
                )
        
        labels_hat = torch.argmax(outputs.logits, dim=1)

        acc = FM.accuracy(labels_hat, labels)

        self.log('val_loss', outputs.loss, on_epoch=True, on_step=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, on_step=True, prog_bar=True, logger=True)
                
        return acc

    def test_step(self, batch, batch_idx):
        acc = self.validation_step(batch, batch_idx)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def train_dataloader(self):
        return self._train_dataloader

    def val_dataloader(self):
        return self._val_dataloader

    def test_dataloader(self):
        return self._test_dataloader

### Use checkpointcallback to save the model with minimal val_loss

In [ ]:
# saves a file like: my/path/albert-openbook-epoch=02-val_loss_epoch=0.32.ckpt
# if you don't want to save checkpoint into google drive, change dirpath!!!
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss_epoch',
    dirpath='/content/drive/My Drive/Albert/Retriever',
    filename='albert-openbook-{epoch:02d}-{val_loss_epoch:.2f}',
    save_top_k=3,
    mode='min',
)

trainer = pl.Trainer(tpu_cores=8, max_epochs=100, callbacks=[checkpoint_callback])

GPU available: False, used: False
TPU available: True, using: 8 TPU cores


In [ ]:
pl_model = Model()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMultipleChoice: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on

### Train the model on TPU

In [ ]:
trainer.fit(pl_model)


  | Name  | Type                    | Params
--------------------------------------------------
0 | model | AlbertForMultipleChoice | 11.7 M
--------------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.737    Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: cleaning up ddp environment...
  warnings.warn(*args, **kwargs)


In [ ]:
%load_ext tensorboard 
%tensorboard --logdir ./lightning_logs

### Test model on test dataset and print accuaracy

In [ ]:
result = trainer.test(test_dataloaders=test_dataloader)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.5806451439857483,
 'test_acc_epoch': 0.6083669662475586,
 'val_acc': 0.5806451439857483,
 'val_acc_epoch': 0.6083669662475586,
 'val_loss': 1.0365203619003296,
 'val_loss_epoch': 0.9303032159805298}
--------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: cleaning up ddp environment...
  warnings.warn(*args, **kwargs)


### Print the best model path with `checkpoint_callback`

In [ ]:
checkpoint_callback.best_model_path

'/content/drive/My Drive/AlbertModel/albert-openbook-epoch=11-val_loss_epoch=0.78.ckpt'

### Save the last model into Google drive and load it again

In [ ]:
trainer.model.model.save_pretrained("/content/drive/My Drive/AlbertModel/lastModel")

In [ ]:
reload_last_model = AlbertForMultipleChoice.from_pretrained("/content/drive/My Drive/AlbertModel/lastModel")

### Load model from checkpoint which is just saved 

In [ ]:
checkpoint = torch.load("/content/drive/My Drive/AlbertModel/albert-openbook-epoch=11-val_loss_epoch=0.78.ckpt")

In [ ]:
# important !!! 
# please read it !!!
# in general you can use the code below to reload the model, but some keys in checkpoint[state_dict'] is a little
# different from the trainer.model.model.state_dict(). So we have to adjust it manually.

# from transformers import AlbertConfig
# config = AlbertConfig.from_pretrained('albert-base-v2')
# m = AlbertForMultipleChoice.from_pretrained(pretrained_model_name_or_path= None, config=config, state_dict=trainer.model.model.state_dict())

new_checkpoint = {}

for key in checkpoint['state_dict'].keys():
  if 'model' in key:
    new_key = key[6:]
    new_checkpoint[new_key] = checkpoint['state_dict'][key]
  else:
    new_checkpoint[key] = checkpoint['state_dict'][key]

In [ ]:
from transformers import AlbertConfig
config = AlbertConfig.from_pretrained('albert-base-v2')
m = AlbertForMultipleChoice.from_pretrained(pretrained_model_name_or_path= None, config=config, state_dict=new_checkpoint)

In [ ]:
from pytorch_lightning.metrics import functional as FM

In [ ]:
class TestModel(pl.LightningModule):

    def __init__(self, model, test_dataloader):
        super(TestModel, self).__init__()

        self.model = model
        self._test_dataloader = test_dataloader

    def test_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        outputs = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
                )
        
        labels_hat = torch.argmax(outputs.logits, dim=1)

        acc = FM.accuracy(labels_hat, labels)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def test_dataloader(self):
        return self._test_dataloader

In [ ]:
trainer_for_test = pl.Trainer(tpu_cores=8)
model_for_test = TestModel(m, test_dataloader)

GPU available: False, used: False
TPU available: True, using: 8 TPU cores


In [ ]:
trainer_for_test.test(model=model_for_test)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.5806451439857483, 'test_acc_epoch': 0.6083669662475586}
--------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: cleaning up ddp environment...
  warnings.warn(*args, **kwargs)


[{'test_acc': 0.5806451439857483, 'test_acc_epoch': 0.6083669662475586}]